In [101]:
import pandas as pd
import numpy as np

## Data 불러오기

In [102]:
hanjeon = pd.read_excel("급속충전이용내역_한전_18.01-19.10.xlsx")
hj = hanjeon.copy()
hj.head(3)

,번호,용도별,충전소,주소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,1,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,17.21,2018-01-01 09:19:36
1,2,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,12.86,2018-01-02 09:16:04
2,3,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,16.10,2018-01-04 14:00:49


In [103]:
len(hj['충전소'].unique())

110

In [104]:
hwangyeong = pd.read_excel("급속충전이용내역_환경부_18.01-19.10.xlsx")
hg = hwangyeong.copy()
hg.head(3)

,번호,충전소대분류,충전소명,주소,설치년도,충전기타입,충전량(kWh),충전시작시간,충전종료시간
0,1,교육문화시설,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,2017,DC차데모+AC3상+DC콤보,7.02,20180101032052,20180101034847
1,2,공공시설,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,2017,DC차데모+AC3상+DC콤보,2.05,20180101041556,20180101042036
2,3,주차시설,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,2017,DC차데모+AC3상+DC콤보,16.95,20180101043214,20180101050354


In [105]:
len(hg['충전소명'].unique())

59

## Data의 Columns명 일치

In [106]:
hj.rename( columns = {
    '충전소 구축일' : '설치년도'
}, inplace = True)

hg.rename( columns = {
    '충전소명' : '충전소',
    '충전시작시간' : '시작일시',
    '충전종료시간' : '종료일시'
}, inplace = True)

In [107]:
hj.head(1)

,번호,용도별,충전소,주소,설치년도,충전구분,충전량(kWh),시작일시
0,1,공용,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,급속,17.21,2018-01-01 09:19:36


In [108]:
hj['구분'] = ['한전'] * len(hj)

In [109]:
hg.head(1)

,번호,충전소대분류,충전소,주소,설치년도,충전기타입,충전량(kWh),시작일시,종료일시
0,1,교육문화시설,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,2017,DC차데모+AC3상+DC콤보,7.02,20180101032052,20180101034847


In [110]:
hg['구분'] = ['환경부'] * len(hg)

In [111]:
hg.rename( columns = {
    '충전소대분류' : '용도별'
}, inplace = True)

## 필요 데이터 추출 및 재구성

### Data 재구성

In [112]:
hj2 = hj.loc[: , ['충전소', '용도별', '주소', '설치년도', '충전량(kWh)', '시작일시', '구분']]
hg2 = hg.loc[: , ['충전소', '용도별', '주소', '설치년도', '충전량(kWh)', '시작일시', '구분']]

In [113]:
df = pd.concat([hj2, hg2], axis = 0)
print(len(df), len(df.columns))
df.head(1)

93943 7


,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전


In [114]:
df['충&주'] = list(map(lambda x, y : x + ' / ' + y, df['충전소'], df['주소']))
print(len(df['충전소'].unique()) , len(df['주소'].unique()), len(df['충&주'].unique()))

169 170 170


In [115]:
df.drop_duplicates(['주소'], keep = 'first').groupby(['충전소']).count()[df.drop_duplicates(['주소'], keep = 'first').groupby(['충전소']).count()['충&주'] >= 2]

,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
충전소,,,,,,,
롯데마트 영종도점,2,2,2,2,2,2,2


### => 충전소는 84개인데 주소가 85개인 이유

In [116]:
df[df['충전소'] == '롯데마트 영종도점']['주소'].unique()

array(['인천광역시 중구 흰바위로 51 , 지상주차장 4층 455구역', '인천광역시 중구 흰바위로 51'],
      dtype=object)

In [117]:
df.loc[df['충전소'] == '롯데마트 영종도점', '주소'] = ['인천광역시 중구 흰바위로 51 지상주차장 4층'] * len(df.loc[df['충전소'] == '롯데마트 영종도점', '주소'])


In [118]:
df[df['충전소'] == '롯데마트 영종도점']['주소'].unique()

array(['인천광역시 중구 흰바위로 51 지상주차장 4층'], dtype=object)

In [119]:
df['충&주'] = list(map(lambda x, y : x + ' / ' + y, df['충전소'], df['주소']))
print(len(df['충전소'].unique()) , len(df['주소'].unique()), len(df['충&주'].unique()))

169 169 169


### => 완료

In [120]:
df.index = range(len(df))

In [121]:
len(df)

93943

In [122]:
df.head(1)

,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


In [123]:
#df.to_csv("19.12.04 충전소 이용내역.csv", encoding = "CP949", index = False)

In [124]:
df2 = df.groupby(['충전소']).sum()
df2

,충전량(kWh)
충전소,
G-Tower(경제자유구역청),30259.44
LH인천본부,63815.83
lh웨스턴블루힐,2270.30
㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65
가정뉴서울아파트,858.89
가좌2동 행정복지센터,7387.11
간석래미안자이,2547.22
갈산역 공영주차장,3879.41
갈산이안 아파트,0.54


In [125]:
df3 = df.drop_duplicates(['충전소'], keep = 'first')
df3.shape

(169, 8)

In [126]:
df3

,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"
504,강화군청,공용,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",2017-09-19,14.74,2018-01-02 10:26:41,한전,"강화군청 / 인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)"
1901,신문리 공영주차장,공용,인천광역시 강화군 강화읍 향나무길 20 (신문리),2018-09-11,9.93,2019-02-01 20:09:50,한전,신문리 공영주차장 / 인천광역시 강화군 강화읍 향나무길 20 (신문리)
2206,온수리 제1공영주차장,공용,인천광역시 강화군 길상면 마니산로 20,2018-09-11,17.46,2019-02-15 17:05:22,한전,온수리 제1공영주차장 / 인천광역시 강화군 길상면 마니산로 20
2304,마니산국민관광지,공용,인천광역시 강화군 화도면 마니산로675번길 18,2017-09-19,25.76,2018-01-05 10:57:57,한전,마니산국민관광지 / 인천광역시 강화군 화도면 마니산로675번길 18
3011,초정마을동아 아파트,아파트용,"인천광역시 계양구 계산새로 109 (용종동, 초정마을동아아파트)",2017-08-12,17.76,2018-08-15 16:04:28,한전,"초정마을동아 아파트 / 인천광역시 계양구 계산새로 109 (용종동, 초정마을동아아파트)"
3040,계양구청,공용,인천광역시 계양구 계산새로 88,2017-11-17,18.88,2018-05-07 14:04:50,한전,계양구청 / 인천광역시 계양구 계산새로 88
4779,홈플러스 작전점,공용,인천광역시 계양구 계양대로 27,2017-11-17,0.44,2018-05-18 12:02:36,한전,홈플러스 작전점 / 인천광역시 계양구 계양대로 27
5459,하나로마트 인천점,공용,인천광역시 계양구 아나지로 341,2017-11-17,0.58,2018-05-18 10:40:46,한전,하나로마트 인천점 / 인천광역시 계양구 아나지로 341
5734,홈플러스 계산점,공용,인천광역시 계양구 오조산공원로 14,2017-11-17,44.85,2018-06-21 13:54:33,한전,홈플러스 계산점 / 인천광역시 계양구 오조산공원로 14


In [127]:
df3.index = range(len(df3))
df4 = df3.iloc[:, [0, 1,2,3,-1,-2]]
df4

,충전소,용도별,주소,설치년도,충&주,구분
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",한전
1,강화군청,공용,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",2017-09-19,"강화군청 / 인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",한전
2,신문리 공영주차장,공용,인천광역시 강화군 강화읍 향나무길 20 (신문리),2018-09-11,신문리 공영주차장 / 인천광역시 강화군 강화읍 향나무길 20 (신문리),한전
3,온수리 제1공영주차장,공용,인천광역시 강화군 길상면 마니산로 20,2018-09-11,온수리 제1공영주차장 / 인천광역시 강화군 길상면 마니산로 20,한전
4,마니산국민관광지,공용,인천광역시 강화군 화도면 마니산로675번길 18,2017-09-19,마니산국민관광지 / 인천광역시 강화군 화도면 마니산로675번길 18,한전
5,초정마을동아 아파트,아파트용,"인천광역시 계양구 계산새로 109 (용종동, 초정마을동아아파트)",2017-08-12,"초정마을동아 아파트 / 인천광역시 계양구 계산새로 109 (용종동, 초정마을동아아파트)",한전
6,계양구청,공용,인천광역시 계양구 계산새로 88,2017-11-17,계양구청 / 인천광역시 계양구 계산새로 88,한전
7,홈플러스 작전점,공용,인천광역시 계양구 계양대로 27,2017-11-17,홈플러스 작전점 / 인천광역시 계양구 계양대로 27,한전
8,하나로마트 인천점,공용,인천광역시 계양구 아나지로 341,2017-11-17,하나로마트 인천점 / 인천광역시 계양구 아나지로 341,한전
9,홈플러스 계산점,공용,인천광역시 계양구 오조산공원로 14,2017-11-17,홈플러스 계산점 / 인천광역시 계양구 오조산공원로 14,한전


In [131]:
df5 = pd.merge(df2, df4, on = ['충전소'])
df5

,충전소,충전량(kWh),용도별,주소,설치년도,충&주,구분
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,한전
1,LH인천본부,63815.83,공용,인천광역시 남동구 논현로46번길 23,2017-01-16,LH인천본부 / 인천광역시 남동구 논현로46번길 23,한전
2,lh웨스턴블루힐,2270.30,아파트용,인천광역시 서구 봉오재2로 37,2018-01-30,lh웨스턴블루힐 / 인천광역시 서구 봉오재2로 37,한전
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,차량정비시설,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,2017,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구 / 인천광역시 서구 원삭로 160 ...,환경부
4,가정뉴서울아파트,858.89,아파트용,인천광역시 서구 봉오대로283번길 6,2017-12-26,가정뉴서울아파트 / 인천광역시 서구 봉오대로283번길 6,한전
5,가좌2동 행정복지센터,7387.11,공공시설,인천광역시 서구 가좌동 30-105,2018,가좌2동 행정복지센터 / 인천광역시 서구 가좌동 30-105,환경부
6,간석래미안자이,2547.22,아파트용,"인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",2018-11-12,"간석래미안자이 / 인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",한전
7,갈산역 공영주차장,3879.41,주차시설,인천광역시 부평구 갈산동 192,2018,갈산역 공영주차장 / 인천광역시 부평구 갈산동 192,환경부
8,갈산이안 아파트,0.54,아파트용,"인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",2017-08-12,"갈산이안 아파트 / 인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",한전
9,강화군청,29291.07,공용,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",2017-09-19,"강화군청 / 인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",한전


In [29]:
#df5.to_csv("19.12.04 충전소 169개 정보.csv", encoding = "CP949", index = False)

## 충전소 고유정보로 이루어진 DataFrame에 총 충전량, 빈도 Column을 추가

### 충전소 위치data

In [128]:
ch_geo = pd.read_csv("19.12.04 충전소 169개 정보_좌표변환.csv", encoding = "CP949")
ch_geo.head(1)

,주소,새우편번호,지번주소 상위부분,지번주소 하위부분,새주소 상위부분,새주소 하위부분,주소정제결과코드,지번코드,새주소 건물관리번호,광역시도코드,...,Y좌표,새주소 길이름,새주소 주건물번호,새주소 부건물번호,구우편번호,도로명길코드,지상지하구분,새주소건물명,블록코드,새주소변환코드
0,인천광역시 연수구 아트센터대로 175,22004.0,인천광역시 연수구 송도동,24-4번지,인천광역시 연수구 아트센터대로,"175, (송도동)",정좌표,2818510600100240004,2818510600100240004000001,28,...,37.395744,아트센터대로,175,0,406740.0,2.818522e+11,0,G-TOWER,NaN,지번확인된 새주소


In [31]:
ch_geo.columns

Index(['주소', '새우편번호', '지번주소 상위부분', '지번주소 하위부분', '새주소 상위부분', '새주소 하위부분',
       '주소정제결과코드', '지번코드', '새주소 건물관리번호', '광역시도코드', '광역시도명', '시군구코드', '시군구명',
       '비자치행정구역코드', '3차행정구역명', '법정동코드', '법정동명', '행정동코드', '행정동명', '리명', '건물명',
       '동명', '호명', '층', '지목', '주번지', '부번지', 'X좌표', 'Y좌표', '새주소 길이름',
       '새주소 주건물번호', '새주소 부건물번호', '구우편번호', '도로명길코드', '지상지하구분', '새주소건물명', '블록코드',
       '새주소변환코드'],
      dtype='object')

In [129]:
ch_geo = ch_geo.loc[:, ['주소', 'X좌표','Y좌표', '시군구명']]

In [33]:
len(ch_geo)

169

In [34]:
len(df)

93943

In [35]:
len(df['충전소'].unique())

169

In [36]:
len(set(ch_geo['주소']) & set(df['주소'].unique()))

169

In [132]:
ch_geo_df = pd.merge(df5, ch_geo, on = ['주소'])
ch_geo_df.head(2)

,충전소,충전량(kWh),용도별,주소,설치년도,충&주,구분,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,한전,126.633795,37.395744,연수구
1,LH인천본부,63815.83,공용,인천광역시 남동구 논현로46번길 23,2017-01-16,LH인천본부 / 인천광역시 남동구 논현로46번길 23,한전,126.712389,37.400860,남동구


In [134]:
sss = ch_geo_df.copy()
sss.groupby(['구분', '시군구명']).size()

구분   시군구명
한전   강화군      5
     계양구      6
     남동구     17
     동구       4
     미추홀구     8
     부평구     10
     서구      23
     연수구     27
     중구      10
환경부  강화군      3
     계양구      5
     남동구      9
     동구       2
     미추홀구     2
     부평구      6
     서구      16
     연수구      7
     옹진군      2
     중구       7
dtype: int64

In [138]:
t = sss.groupby(['구분', '시군구명']).size()
t = pd.DataFrame(t)
t

0
구분  시군구명    
한전  강화군    5
    계양구    6
    남동구   17
    동구     4
    미추홀구   8
    부평구   10
    서구    23
    연수구   27
    중구    10
환경부 강화군    3
    계양구    5
    남동구    9
    동구     2
    미추홀구   2
    부평구    6
    서구    16
    연수구    7
    옹진군    2
    중구     7

In [154]:
t.columns = ['충전기']
t1 = [ '한전'] *9 + ['환경부'] * 10
t2 = [ '강화군', '계양구', '남동구', '동구', '미추홀구', '부평구', '서구',
          '연수구', '중구',  '강화군', '계양구', '남동구', '동구', '미추홀구', '부평구', '서구',
          '연수구', '옹진군', '중구' ]
t3 = t['충전기']

tot = pd.DataFrame( [t1,t2,t3], index = ['구분', '군구', '충전기']).T
tot

,구분,군구,충전기
0,한전,강화군,5
1,한전,계양구,6
2,한전,남동구,17
3,한전,동구,4
4,한전,미추홀구,8
5,한전,부평구,10
6,한전,서구,23
7,한전,연수구,27
8,한전,중구,10
9,환경부,강화군,3


In [156]:
tot.to_excel("구분별 군구별 충전기수.xlsx", encoding = "CP949", index = False)
tot.to_csv("구분별 군구별 충전기수.csv", encoding = "CP949", index = False)

In [38]:
#ch_geo_df.to_csv("19.12.04 충전소 정보 및 좌표.csv", encoding = "CP949", index = False)

In [39]:
ch_geo2 = ch_geo_df[ ch_geo_df['충전소'].isin(list(df5['충전소'].unique())) ]
ch_geo2.head(1)

,충전소,충전량(kWh),용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


### df에 좌표와 시군구명 추가

In [40]:
print(len(df), len(df.columns), len(df['충전소'].unique()))
df.head(1)

93943 8 169


,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


In [41]:
print(len(ch_geo2), len(ch_geo2.columns))
ch_geo2.head(1)

169 9


,충전소,충전량(kWh),용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


In [42]:
ch_geo2.loc[ch_geo2['충전소'] == '롯데마트 영종도점', '주소'] = '인천광역시 중구 흰바위로 51 지상주차장 4층'

In [43]:
ch_geo2[ch_geo2['충전소'] == '롯데마트 영종도점']
ch_geo2 = ch_geo2.drop_duplicates(['주소'], keep = 'first')
ch_geo2.index = range(len(ch_geo2))
ch_geo2.head(1)

,충전소,충전량(kWh),용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


In [44]:
len(ch_geo2)

169

### 충전이력 빈도

In [45]:
df.groupby(['충전소']).count()

,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
충전소,,,,,,,
G-Tower(경제자유구역청),1261,1261,1261,1261,1261,1261,1261
LH인천본부,3078,3078,3078,3078,3078,3078,3078
lh웨스턴블루힐,83,83,83,83,83,83,83
㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,651,651,651,651,651,651,651
가정뉴서울아파트,41,41,41,41,41,41,41
가좌2동 행정복지센터,400,400,400,400,400,400,400
간석래미안자이,94,94,94,94,94,94,94
갈산역 공영주차장,187,187,187,187,187,187,187
갈산이안 아파트,1,1,1,1,1,1,1


### 충전소별 충전 일수

In [46]:
df.head(1)

,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)"


In [47]:
df.loc[2, '시작일시']

'2018-01-04 14:00:49'

In [48]:
df2 = df.copy()
df2['충전일'] = list(map(lambda x : str(x) , df2['시작일시']))
df2['충전일'] = list(map(lambda x : x.replace('-', ''), df2['충전일']))
df2['충전일'] = list(map(lambda x : x[2:8] , df2['충전일']))
df2['충전일'] = list(map(lambda x : int(x) , df2['충전일']))
df2.head(1)

,충전소,용도별,주소,설치년도,충전량(kWh),시작일시,구분,충&주,충전일
0,강화지사,공용,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",2017-02-27,17.21,2018-01-01 09:19:36,한전,"강화지사 / 인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",180101


In [49]:
temp = df2.sort_values(['충전소', '충전일'], ascending = True).loc[:,['충전소','충전일']]
temp.head(3)

,충전소,충전일
26726,G-Tower(경제자유구역청),180423
26727,G-Tower(경제자유구역청),180423
26728,G-Tower(경제자유구역청),180424


In [50]:
temp['충전소'].unique()

array(['G-Tower(경제자유구역청)', 'LH인천본부', 'lh웨스턴블루힐',
       '㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구', '가정뉴서울아파트', '가좌2동 행정복지센터', '간석래미안자이',
       '갈산역 공영주차장', '갈산이안 아파트', '강화군청', '강화지사', '강화풍물시장 공영주차장',
       '검단2공영주차장', '검단아이파크아파트', '검단자이1단지 아파트', '검암2차풍림아파트', '계산1택지 공영주차장',
       '계양구 공영주차장', '계양구청', '관교쌍용 아파트', '구월 아시아드선수촌 1단지', '구월아시아드선수촌6단지',
       '구월아시아드선수촌센트럴자이 아파트', '구월힐스테이트1단지', '그린애비뉴7단지', '남동복지관', '남인천지사',
       '논현힐스테이트 아파트', '늘솔길공원 3주차장', '더샵퍼스트월드', '덕산국민여가캠핑장',
       '도원역 부근 공영주차장', '동구청앞 공영주차장', '동산휴먼시아1단지 아파트', '롯데마트 검단점',
       '롯데마트 부평역점', '롯데마트 연수점', '롯데마트 영종도점', '롯데마트 청라점', '롯데캐슬 캠퍼스타운',
       '마니산국민관광지', '모탱이공원', '미래타운4단지', '부원 공영주차장', '부평국민체육센터',
       '부평깡시장 공영주차장', '부평아트센터', '산곡우성1.2.3차 아파트', '산곡한신휴아파트', '삼산농산물도매시장',
       '삼산서해그랑블 아파트', '서곶근린공원 제1주차장', '서림초교 뒤 공영주차장', '서운체육공원 주차장',
       '서창 LH3단지 청아안', '서창 서해그랑블', '서창LH6단지', '서창LH8단지 아파트', '서창현대모닝사이드',
       '소래제1주차장', '송도골프', '송도글로벌캠퍼스푸르지오 아파트', '송도더샵그린애비뉴8단지',
       '송도더샵그린워크1차', '송도더샵그린워크2차 아파트', '송도더샵그린워크3차1

In [51]:
temp.groupby(['충전소','충전일']).count().index.levels[0][0]

'G-Tower(경제자유구역청)'

In [52]:
temp.groupby(['충전소', '충전일']).count().index.levels

FrozenList([['G-Tower(경제자유구역청)', 'LH인천본부', 'lh웨스턴블루힐', '㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구', '가정뉴서울아파트', '가좌2동 행정복지센터', '간석래미안자이', '갈산역 공영주차장', '갈산이안 아파트', '강화군청', '강화지사', '강화풍물시장 공영주차장', '검단2공영주차장', '검단아이파크아파트', '검단자이1단지 아파트', '검암2차풍림아파트', '계산1택지 공영주차장', '계양구 공영주차장', '계양구청', '관교쌍용 아파트', '구월 아시아드선수촌 1단지', '구월아시아드선수촌6단지', '구월아시아드선수촌센트럴자이 아파트', '구월힐스테이트1단지', '그린애비뉴7단지', '남동복지관', '남인천지사', '논현힐스테이트 아파트', '늘솔길공원 3주차장', '더샵퍼스트월드', '덕산국민여가캠핑장', '도원역 부근 공영주차장', '동구청앞 공영주차장', '동산휴먼시아1단지 아파트', '롯데마트 검단점', '롯데마트 부평역점', '롯데마트 연수점', '롯데마트 영종도점', '롯데마트 청라점', '롯데캐슬 캠퍼스타운', '마니산국민관광지', '모탱이공원', '미래타운4단지', '부원 공영주차장', '부평국민체육센터', '부평깡시장 공영주차장', '부평아트센터', '산곡우성1.2.3차 아파트', '산곡한신휴아파트', '삼산농산물도매시장', '삼산서해그랑블 아파트', '서곶근린공원 제1주차장', '서림초교 뒤 공영주차장', '서운체육공원 주차장', '서창 LH3단지 청아안', '서창 서해그랑블', '서창LH6단지', '서창LH8단지 아파트', '서창현대모닝사이드', '소래제1주차장', '송도골프', '송도글로벌캠퍼스푸르지오 아파트', '송도더샵그린애비뉴8단지', '송도더샵그린워크1차', '송도더샵그린워크2차 아파트', '송도더샵그린워크3차17블럭', '송도더샵그린워크3차18블럭', '송도더샵센트럴파크2차아파트', '송도더샵하버뷰13단지', '송도롯데캐슬 아파트', '송도성지리벨루스', '송도아트윈푸르지

In [53]:
s = list(temp['충전소'].unique())
ch_day_cont = []

for i in s :
    ch_day_cont.append(len(temp[temp['충전소'] == i]['충전일'].unique()))
    
len(ch_day_cont)

169

### 총충전량

In [54]:
df.groupby(['충전소']).sum()

,충전량(kWh)
충전소,
G-Tower(경제자유구역청),30259.44
LH인천본부,63815.83
lh웨스턴블루힐,2270.30
㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65
가정뉴서울아파트,858.89
가좌2동 행정복지센터,7387.11
간석래미안자이,2547.22
갈산역 공영주차장,3879.41
갈산이안 아파트,0.54


In [55]:
ch_tot = df.groupby(['충전소']).sum()
ch_tot['충전소'] = ch_tot.index
ch_tot.index = range(len(ch_tot))
ch_tot = ch_tot.iloc[:, [1,0]]
ch_tot.head(10)

,충전소,충전량(kWh)
0,G-Tower(경제자유구역청),30259.44
1,LH인천본부,63815.83
2,lh웨스턴블루힐,2270.30
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65
4,가정뉴서울아파트,858.89
5,가좌2동 행정복지센터,7387.11
6,간석래미안자이,2547.22
7,갈산역 공영주차장,3879.41
8,갈산이안 아파트,0.54
9,강화군청,29291.07


In [56]:
ch_tot['충전일수'] = ch_day_cont
ch_tot.head(5)

,충전소,충전량(kWh),충전일수
0,G-Tower(경제자유구역청),30259.44,408
1,LH인천본부,63815.83,612
2,lh웨스턴블루힐,2270.30,74
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,375
4,가정뉴서울아파트,858.89,40


## 충전소 위치데이터와 충전 일수와 총 충전량 결합

In [57]:
ch_tot

,충전소,충전량(kWh),충전일수
0,G-Tower(경제자유구역청),30259.44,408
1,LH인천본부,63815.83,612
2,lh웨스턴블루힐,2270.30,74
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,375
4,가정뉴서울아파트,858.89,40
5,가좌2동 행정복지센터,7387.11,196
6,간석래미안자이,2547.22,84
7,갈산역 공영주차장,3879.41,131
8,갈산이안 아파트,0.54,1
9,강화군청,29291.07,517


In [58]:
ch_geo2

,충전소,충전량(kWh),용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구
1,LH인천본부,63815.83,공용,인천광역시 남동구 논현로46번길 23,2017-01-16,LH인천본부 / 인천광역시 남동구 논현로46번길 23,126.712389,37.400860,남동구
2,lh웨스턴블루힐,2270.30,아파트용,인천광역시 서구 봉오재2로 37,2018-01-30,lh웨스턴블루힐 / 인천광역시 서구 봉오재2로 37,126.666490,37.534587,서구
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,10538.65,차량정비시설,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,2017,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구 / 인천광역시 서구 원삭로 160 ...,126.666335,37.563021,서구
4,가정뉴서울아파트,858.89,아파트용,인천광역시 서구 봉오대로283번길 6,2017-12-26,가정뉴서울아파트 / 인천광역시 서구 봉오대로283번길 6,126.682082,37.524914,서구
5,가좌2동 행정복지센터,7387.11,공공시설,인천광역시 서구 가좌동 30-105,2018,가좌2동 행정복지센터 / 인천광역시 서구 가좌동 30-105,126.684851,37.490841,서구
6,간석래미안자이,2547.22,아파트용,"인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",2018-11-12,"간석래미안자이 / 인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",126.708967,37.460147,남동구
7,갈산역 공영주차장,3879.41,주차시설,인천광역시 부평구 갈산동 192,2018,갈산역 공영주차장 / 인천광역시 부평구 갈산동 192,126.723488,37.515556,부평구
8,갈산이안 아파트,0.54,아파트용,"인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",2017-08-12,"갈산이안 아파트 / 인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",126.722401,37.516932,부평구
9,강화군청,29291.07,공용,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",2017-09-19,"강화군청 / 인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",126.487733,37.746131,강화군


In [59]:
df2 = pd.merge( ch_tot, ch_geo2 , how = 'left', on = ['충전소'])
print( len(df2), len(df2.columns) )
df2.head(1)

169 11


,충전소,충전량(kWh)_x,충전일수,충전량(kWh)_y,용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,408,30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


In [60]:
#df2.to_csv("2019.12.04 충전소 정보 및 좌표.csv", encoding = "CP949", index = False)

In [61]:
df2.head(1)

,충전소,충전량(kWh)_x,충전일수,충전량(kWh)_y,용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,408,30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


In [62]:
df2.groupby(['시군구명']).size()

시군구명
강화군      8
계양구     11
남동구     26
동구       6
미추홀구    10
부평구     16
서구      39
연수구     34
옹진군      2
중구      17
dtype: int64

In [63]:
df2.head(1)

,충전소,충전량(kWh)_x,충전일수,충전량(kWh)_y,용도별,주소,설치년도,충&주,X좌표,Y좌표,시군구명
0,G-Tower(경제자유구역청),30259.44,408,30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구


## 버퍼 km별 전기차수

In [64]:
km_ele = pd.read_csv("충전소명_주소_좌표변환_버퍼km별_전기차수.csv",encoding = "CP949")
km_ele.head(3)

,충전소,주소,시군구명,X좌표,Y좌표,CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,44,75,136,197,273
1,인천 학익풍림아이원 아파트,인천광역시 미추홀구 매소홀로 340,미추홀구,126.66147,37.44042,9,61,163,250,393
2,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,29,121,208,320,371


## df2와 km_ele join

In [65]:
df3 = pd.merge(df2, km_ele, on = '충전소')
df3.head(3)

,충전소,충전량(kWh)_x,충전일수,충전량(kWh)_y,용도별,주소_x,설치년도,충&주,X좌표_x,Y좌표_x,시군구명_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,CarNum_2km,CarNum_3km,CarNum_4km,CarNum_5km
0,G-Tower(경제자유구역청),30259.44,408,30259.44,공용,인천광역시 연수구 아트센터대로 175,2017-11-17,G-Tower(경제자유구역청) / 인천광역시 연수구 아트센터대로 175,126.633795,37.395744,연수구,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,127,213,288,358
1,LH인천본부,63815.83,612,63815.83,공용,인천광역시 남동구 논현로46번길 23,2017-01-16,LH인천본부 / 인천광역시 남동구 논현로46번길 23,126.712389,37.400860,남동구,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16,83,110,174,275
2,lh웨스턴블루힐,2270.30,74,2270.30,아파트용,인천광역시 서구 봉오재2로 37,2018-01-30,lh웨스턴블루힐 / 인천광역시 서구 봉오재2로 37,126.666490,37.534587,서구,인천광역시 서구 봉오재2로 37,서구,126.66649,37.53459,22,110,155,207,309


## 총충전량과 km별 버퍼 내에 전기차수 상관관계

In [66]:
df3_pre = df3.iloc[:, [0,2,1, 11,12,13,14,15]]
df3_pre

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km
0,G-Tower(경제자유구역청),408,30259.44,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56
1,LH인천본부,612,63815.83,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16
2,lh웨스턴블루힐,74,2270.30,인천광역시 서구 봉오재2로 37,서구,126.66649,37.53459,22
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,서구,126.66633,37.56302,20
4,가정뉴서울아파트,40,858.89,인천광역시 서구 봉오대로283번길 6,서구,126.68208,37.52491,4
5,가좌2동 행정복지센터,196,7387.11,인천광역시 서구 가좌동 30-105,서구,126.68485,37.49084,6
6,간석래미안자이,84,2547.22,"인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",남동구,126.70897,37.46015,39
7,갈산역 공영주차장,131,3879.41,인천광역시 부평구 갈산동 192,부평구,126.72349,37.51556,92
8,갈산이안 아파트,1,0.54,"인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",부평구,126.72240,37.51693,61
9,강화군청,517,29291.07,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",강화군,126.48773,37.74613,10


In [67]:
df3_pre.iloc[:,2:].corr()

,충전량(kWh)_x,X좌표_y,Y좌표_y,CarNum_1km
충전량(kWh)_x,1.000000,-0.024762,0.031119,0.022684
X좌표_y,-0.024762,1.000000,-0.256892,0.281332
Y좌표_y,0.031119,-0.256892,1.000000,-0.299735
CarNum_1km,0.022684,0.281332,-0.299735,1.000000


## 총충전량/충전일수 와 거리별 버퍼내 전기차수 상관관계

In [68]:
df3_pre['일수당충전량'] = df3_pre['충전량(kWh)_x'] / df3_pre['충전일수']
df3_pre.head(3)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,74.165294
1,LH인천본부,612,63815.83,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16,104.274232
2,lh웨스턴블루힐,74,2270.30,인천광역시 서구 봉오재2로 37,서구,126.66649,37.53459,22,30.679730


In [69]:
df3_pre['일수당충전량'].isnull().sum()

0

In [70]:
df3_pre.iloc[:,3:].corr()

,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
X좌표_y,1.000000,-0.256892,0.281332,0.023488
Y좌표_y,-0.256892,1.000000,-0.299735,-0.016396
CarNum_1km,0.281332,-0.299735,1.000000,0.054400
일수당충전량,0.023488,-0.016396,0.054400,1.000000


In [71]:
pd.set_option("display.max_rows", 100)
df3_pre['충전소']

0                G-Tower(경제자유구역청)
1                          LH인천본부
2                        lh웨스턴블루힐
3      ㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구
4                        가정뉴서울아파트
5                     가좌2동 행정복지센터
6                         간석래미안자이
7                       갈산역 공영주차장
8                        갈산이안 아파트
9                            강화군청
10                           강화지사
11                   강화풍물시장 공영주차장
12                       검단2공영주차장
13                      검단아이파크아파트
14                    검단자이1단지 아파트
15                      검암2차풍림아파트
16                    계산1택지 공영주차장
17                      계양구 공영주차장
18                           계양구청
19                       관교쌍용 아파트
20                 구월 아시아드선수촌 1단지
21                   구월아시아드선수촌6단지
22             구월아시아드선수촌센트럴자이 아파트
23                     구월힐스테이트1단지
24                       그린애비뉴7단지
25                          남동복지관
26                          남인천지사
27                    논현힐스테이트 아파트
28                     늘솔길공원 3주차장
29            

In [72]:
df3_pre[ df3_pre['충전소'].isin(['마트', '홈플러스'])]

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량


In [73]:
mart_li = []

In [74]:
df3_pre = df3_pre.drop_duplicates(['충전소'], keep = 'first').sort_values(['충전소'])
df3_pre.index = range(len(df3_pre))
df3_pre

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,74.165294
1,LH인천본부,612,63815.83,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16,104.274232
2,lh웨스턴블루힐,74,2270.30,인천광역시 서구 봉오재2로 37,서구,126.66649,37.53459,22,30.679730
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,서구,126.66633,37.56302,20,28.103067
4,가정뉴서울아파트,40,858.89,인천광역시 서구 봉오대로283번길 6,서구,126.68208,37.52491,4,21.472250
5,가좌2동 행정복지센터,196,7387.11,인천광역시 서구 가좌동 30-105,서구,126.68485,37.49084,6,37.689337
6,간석래미안자이,84,2547.22,"인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",남동구,126.70897,37.46015,39,30.324048
7,갈산역 공영주차장,131,3879.41,인천광역시 부평구 갈산동 192,부평구,126.72349,37.51556,92,29.613817
8,갈산이안 아파트,1,0.54,"인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",부평구,126.72240,37.51693,61,0.540000
9,강화군청,517,29291.07,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",강화군,126.48773,37.74613,10,56.655841


In [75]:
mart_li = []

for i in list(df3_pre['충전소']) :
    if( (i.find('마트') != -1 ) | (i.find('홈플러스') != -1)) : 
        mart_li.append(i)
mart_li

['롯데마트 검단점',
 '롯데마트 부평역점',
 '롯데마트 연수점',
 '롯데마트 영종도점',
 '롯데마트 청라점',
 '이마트 동인천점',
 '이마트 연수점',
 '하나로마트 인천점',
 '홈플러스 가좌점',
 '홈플러스 계산점',
 '홈플러스 구월점',
 '홈플러스 인천연수점',
 '홈플러스 인하점',
 '홈플러스 작전점']

In [76]:
df3_mart = df3_pre[df3_pre['충전소'].isin(mart_li)]

In [77]:
df3_Nomart = df3_pre[df3_pre['충전소'].isin(list(set(df3_pre['충전소']) - set(mart_li)))]

In [78]:
df3_mart

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
34,롯데마트 검단점,499,28768.72,인천광역시 서구 원당대로 581 (마전동),서구,126.66457,37.59453,16,57.652745
35,롯데마트 부평역점,501,20124.35,인천광역시 부평구 광장로 16 (부평동),부평구,126.72335,37.48957,19,40.168363
36,롯데마트 연수점,335,16488.97,"인천광역시 연수구 청능대로 87, 2층 206",연수구,126.67517,37.41512,46,49.220806
37,롯데마트 영종도점,516,29106.06,"인천광역시 중구 흰바위로 51 , 지상주차장 4층 455구역",중구,126.49121,37.49267,16,56.407093
38,롯데마트 청라점,524,51481.52,인천광역시 서구 청라 커낼로 252(경서동),서구,126.63260,37.53466,25,98.247176
110,이마트 동인천점,235,17285.21,인천광역시 중구 인중로 134,중구,126.62917,37.46655,9,73.554085
111,이마트 연수점,356,7994.79,"인천광역시 연수구 경원대로 184 , 주차장 옥상",연수구,126.68112,37.40422,37,22.457275
156,하나로마트 인천점,208,6853.27,인천광역시 계양구 아나지로 341,계양구,126.73058,37.52606,13,32.948413
163,홈플러스 가좌점,126,1813.40,"인천광역시 서구 가정로151번길 11 , 지하주차장 1층 M0구역",서구,126.67070,37.49761,44,14.392063
164,홈플러스 계산점,318,11688.85,인천광역시 계양구 오조산공원로 14,계양구,126.73618,37.53947,90,36.757390


In [79]:
df3_Nomart

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,74.165294
1,LH인천본부,612,63815.83,인천광역시 남동구 논현로46번길 23,남동구,126.71239,37.40086,16,104.274232
2,lh웨스턴블루힐,74,2270.30,인천광역시 서구 봉오재2로 37,서구,126.66649,37.53459,22,30.679730
3,㈜현대자동차 블루핸즈 서인천 서비스 주차장 입구,375,10538.65,인천광역시 서구 원삭로 160 주차장 입구 우측 완속충전기 옆,서구,126.66633,37.56302,20,28.103067
4,가정뉴서울아파트,40,858.89,인천광역시 서구 봉오대로283번길 6,서구,126.68208,37.52491,4,21.472250
5,가좌2동 행정복지센터,196,7387.11,인천광역시 서구 가좌동 30-105,서구,126.68485,37.49084,6,37.689337
6,간석래미안자이,84,2547.22,"인천광역시 남동구 남동대로 860 (간석동, 간석래미안자이아파트)",남동구,126.70897,37.46015,39,30.324048
7,갈산역 공영주차장,131,3879.41,인천광역시 부평구 갈산동 192,부평구,126.72349,37.51556,92,29.613817
8,갈산이안 아파트,1,0.54,"인천광역시 부평구 평천로305번길 15 (갈산동, 갈산이안아파트)",부평구,126.72240,37.51693,61,0.540000
9,강화군청,517,29291.07,"인천광역시 강화군 강화읍 강화대로 394 (관청리, 강화군청)",강화군,126.48773,37.74613,10,56.655841


In [80]:
print(len(df3_pre), len(df3_mart), len(df3_Nomart), len(df3_mart) + len(df3_Nomart))

169 14 155 169


## 대형마트와 아닌 곳 km별 버퍼 내에 전기차수 상관관계

In [81]:
df3_mart.iloc[:, 2:].corr()

,충전량(kWh)_x,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
충전량(kWh)_x,1.000000,-0.338036,0.142191,-0.179858,0.928805
X좌표_y,-0.338036,1.000000,0.059302,0.356623,-0.258770
Y좌표_y,0.142191,0.059302,1.000000,-0.086931,0.045459
CarNum_1km,-0.179858,0.356623,-0.086931,1.000000,-0.197264
일수당충전량,0.928805,-0.258770,0.045459,-0.197264,1.000000


In [82]:
df3_Nomart.iloc[:, 2:].corr()

,충전량(kWh)_x,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
충전량(kWh)_x,1.000000,0.006072,0.021010,0.046445,0.868219
X좌표_y,0.006072,1.000000,-0.276089,0.271718,0.053598
Y좌표_y,0.021010,-0.276089,1.000000,-0.313844,-0.025240
CarNum_1km,0.046445,0.271718,-0.313844,1.000000,0.087267
일수당충전량,0.868219,0.053598,-0.025240,0.087267,1.000000


In [83]:
df3_pre.head(1)

,충전소,충전일수,충전량(kWh)_x,주소_y,시군구명_y,X좌표_y,Y좌표_y,CarNum_1km,일수당충전량
0,G-Tower(경제자유구역청),408,30259.44,인천광역시 연수구 아트센터대로 175,연수구,126.63379,37.39574,56,74.165294


In [84]:
df_q = pd.read_csv("버퍼2km_qgis_join.csv", encoding = "CP949")
df_q.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,경제인구,변압기,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,0.0,0.0,2761.0,24.0,27526.0,67.0


In [85]:
df_q = pd.merge(df_q, df3_pre.iloc[:,[0,1,2,8]], left_on = '충전소_명', right_on = '충전소')
df_q

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,경제인구,변압기,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,충전소,충전일수,충전량(kWh)_x,일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,0.0,0.0,2761.0,24.0,27526.0,67.0,인천서구청 제1주차장,206,3224.460,15.652718
1,인천 학익풍림아이원 아파트,인천광역시 미추홀구 매소홀로 340,미추홀구,126.66147,37.44042,61,0.0,0.0,0.0,0.0,10453.0,25.0,87364.0,105.0,인천 학익풍림아이원 아파트,284,7390.680,26.023521
2,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,121,0.0,0.0,0.0,0.0,2597.0,26.0,62531.0,38.0,연세대학교 국제캠퍼스,656,53172.870,81.056204
3,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,남동구,126.69954,37.45113,103,0.0,0.0,0.0,0.0,10903.0,65.0,17727.0,101.0,중부지방고용노동청 별관 앞,650,53635.050,82.515462
4,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,강화군,126.49287,37.74142,21,0.0,0.0,0.0,0.0,146.0,2.0,659.0,5.0,강화풍물시장 공영주차장,597,27766.940,46.510787
5,영종대교 휴게소 주차장,인천광역시 서구 경서동 산 257-4 지상 버스주차장 옆,서구,126.60708,37.55390,1,0.0,0.0,0.0,0.0,463.0,3.0,NaN,NaN,영종대교 휴게소 주차장,578,30137.350,52.140744
6,더샵퍼스트월드,인천광역시 연수구 해돋이로 107,연수구,126.64765,37.39279,186,0.0,0.0,0.0,0.0,6500.0,43.0,104796.0,75.0,더샵퍼스트월드,262,5682.960,21.690687
7,인천공항 정부합동청사,"인천광역시 중구 공항로424번길 47 , 정문 주차장",중구,126.46134,37.44039,10,0.0,0.0,0.0,0.0,NaN,NaN,295.0,1.0,인천공항 정부합동청사,437,8148.950,18.647483
8,검단2공영주차장,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,서구,126.67502,37.59202,36,0.0,0.0,0.0,0.0,955.0,8.0,6599.0,15.0,검단2공영주차장,630,44358.600,70.410476
9,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",강화군,126.49913,37.74187,18,0.0,0.0,0.0,0.0,88.0,1.0,659.0,5.0,강화지사,342,10877.100,31.804386


In [86]:
df_q2 = pd.read_csv("버퍼2km_qgis_join_수정.csv", encoding = "CP949")
df_q2

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331
1,인천 학익풍림아이원 아파트,인천광역시 미추홀구 매소홀로 340,미추홀구,126.66147,37.44042,61,0.0,0.0,10453.0,25.0,87364.0,105.0,144322.0,5,45,641
2,연세대학교 국제캠퍼스,인천광역시 연수구 송도동 162-1 포스코 그린빌딩 앞 주차장,연수구,126.66727,37.38151,121,0.0,0.0,2597.0,26.0,62531.0,38.0,59740.0,2,20,317
3,중부지방고용노동청 별관 앞,인천광역시 남동구 구월3동 1111-6 중부지방고용노동청 별관 앞,남동구,126.69954,37.45113,103,0.0,0.0,10903.0,65.0,17727.0,101.0,232312.0,3,68,1383
4,강화풍물시장 공영주차장,인천광역시 강화군 강화읍 갑곳리 849 고엽제전우회 건물 옆,강화군,126.49287,37.74142,21,0.0,0.0,146.0,2.0,659.0,5.0,14641.0,1,17,145
5,영종대교 휴게소 주차장,인천광역시 서구 경서동 산 257-4 지상 버스주차장 옆,서구,126.60708,37.55390,1,0.0,0.0,463.0,3.0,NaN,NaN,1193.0,0,1,63
6,더샵퍼스트월드,인천광역시 연수구 해돋이로 107,연수구,126.64765,37.39279,186,0.0,0.0,6500.0,43.0,104796.0,75.0,86413.0,3,27,358
7,인천공항 정부합동청사,"인천광역시 중구 공항로424번길 47 , 정문 주차장",중구,126.46134,37.44039,10,0.0,0.0,NaN,NaN,295.0,1.0,488.0,0,0,30
8,검단2공영주차장,인천광역시 서구 당하동 1097-5 검단4동 주민센터 옆,서구,126.67502,37.59202,36,0.0,0.0,955.0,8.0,6599.0,15.0,68579.0,9,17,421
9,강화지사,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",강화군,126.49913,37.74187,18,0.0,0.0,88.0,1.0,659.0,5.0,13732.0,1,14,138


In [87]:
df_q3 = pd.merge(df_q2, df3_pre.iloc[:,[0,1,2,8]], left_on = '충전소_명', right_on = '충전소')
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기,충전소,충전일수,충전량(kWh)_x,일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,0.0,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331,인천서구청 제1주차장,206,3224.46,15.652718


In [88]:
df_q3['관공서'] = df_q3['도서관'] + df_q3['민행기'] 

In [89]:
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일수,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,도서관,민행기,변압기,충전소,충전일수,충전량(kWh)_x,일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,0.0,2761.0,24.0,27526.0,67.0,66686.0,9,20,331,인천서구청 제1주차장,206,3224.46,15.652718


In [90]:
del df_q3['충전소']
del df_q3['일수']
del df_q3['도서관']
del df_q3['민행기']
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh)_x,일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,2761.0,24.0,27526.0,67.0,66686.0,331,206,3224.46,15.652718


In [91]:
len(df_q3)

170

In [92]:
df_q3[df_q3['충전소_명'] == '롯데마트 영종도점']['주소'] = ['인천광역시 중구 흰바위로 51 지상주차장 4층'] * 2

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
len(df_q3)

170

In [94]:
df_q3 = df_q3.drop_duplicates(['충전소_명'], keep = 'first')

In [95]:
len(df_q3)

169

In [96]:
df_q3.head(1)

,충전소_명,주소,시군구명,X좌표,Y좌표,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh)_x,일수당충전량
0,인천서구청 제1주차장,"인천광역시 서구 탁옥로51번길 26 , 서구청후문 주차타워 지상 1층",서구,126.67598,37.54606,75,29,2761.0,24.0,27526.0,67.0,66686.0,331,206,3224.46,15.652718


In [97]:
df_q3.isnull().sum()

충전소_명          0
주소             0
시군구명           0
X좌표            0
Y좌표            0
car_num_2k     0
관공서            0
일반주차면          8
일반cnt          8
집합주차면         10
집합cnt         10
경제인구           0
변압기            0
충전일수           0
충전량(kWh)_x     0
일수당충전량         0
dtype: int64

In [98]:
df_q3 = df_q3.fillna(0)

# 2km 버퍼 내에 Columns들 수 간의 상관관계

In [99]:
df_q3.iloc[:, 5:].corr()

,car_num_2k,관공서,일반주차면,일반cnt,집합주차면,집합cnt,경제인구,변압기,충전일수,충전량(kWh)_x,일수당충전량
car_num_2k,1.000000,0.517507,0.530444,0.688203,0.361565,0.318108,0.549791,0.428657,-0.003233,0.079919,0.116101
관공서,0.517507,1.000000,0.706968,0.748227,0.326870,0.455556,0.878811,0.866190,0.097879,0.142033,0.129420
일반주차면,0.530444,0.706968,1.000000,0.872842,0.281213,0.371849,0.600760,0.707121,0.031970,0.100909,0.128147
일반cnt,0.688203,0.748227,0.872842,1.000000,0.489099,0.572928,0.707400,0.820720,0.100467,0.200444,0.235659
집합주차면,0.361565,0.326870,0.281213,0.489099,1.000000,0.812653,0.442438,0.414395,0.031410,0.053453,0.044608
집합cnt,0.318108,0.455556,0.371849,0.572928,0.812653,1.000000,0.506159,0.576976,0.114716,0.145685,0.157346
경제인구,0.549791,0.878811,0.600760,0.707400,0.442438,0.506159,1.000000,0.849129,0.085793,0.127749,0.108425
변압기,0.428657,0.866190,0.707121,0.820720,0.414395,0.576976,0.849129,1.000000,0.132749,0.199174,0.187455
충전일수,-0.003233,0.097879,0.031970,0.100467,0.031410,0.114716,0.085793,0.132749,1.000000,0.873204,0.658479
충전량(kWh)_x,0.079919,0.142033,0.100909,0.200444,0.053453,0.145685,0.127749,0.199174,0.873204,1.000000,0.877543
